## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent on Covered Strangles
            </span>
        </td>
    </tr>
</table>

In [1]:
# Imports

from agents import Agent, WebSearchTool, trace, Runner, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from dotenv import load_dotenv
import asyncio
import os
from IPython.display import display, Markdown
from pprint import pprint
import requests
load_dotenv(override=True)

# Constants

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern.

Costs are here: https://platform.openai.com/docs/pricing#web-search

## We will be making 4 Agents:

1. Search Agent - searches online given a search term using an OpenAI hosted tool
2. Planner Agent - given a query from the user, come up with searches
3. Report Agent - make a report on results
4. Push Agent - send a notification to the user's phone with a summary

## Our First Agent: Search Agent

Given a Search term, search for it on the internet and summarize results.

In [2]:
INSTRUCTIONS = "You are a research assistant. For a Crypto Symbol Find Support and Resistances for 1 week, 30 days , 60 days \
    Scavenge, reddit, news all top crypto forums to get Supports and resistances and output them. "

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4.1-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [3]:
message = "Whats the forecast for ETH?"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of May 17, 2025, Ethereum (ETH) is trading at $2,505.00 USD, reflecting a slight decrease of 2.57% from the previous close.

## Stock market information for Ethereum (ETH)
- Ethereum is a crypto in the CRYPTO market.
- The price is 2505.0 USD currently with a change of -65.94 USD (-0.03%) from the previous close.
- The intraday high is 2631.38 USD and the intraday low is 2466.4 USD.


Looking ahead, various analysts have provided forecasts for ETH's price by the end of 2025:

- **Standard Chartered Bank** has revised its prediction, now anticipating ETH to reach $4,000 by 2025, down from an earlier estimate of $10,000. This adjustment considers the impact of Layer-2 solutions on Ethereum's scalability and transaction costs. ([fxstreet.com](https://www.fxstreet.com/cryptocurrencies/news/ethereum-price-forecast-eth-consolidates-below-2-000-as-standard-chartered-alters-its-prediction-for-2025-202503180509?utm_source=openai))

- **Bitnation**, a crypto forecasting platform, projects an average ETH price of $6,842.76 by 2025, with potential for further growth in subsequent years. ([benzinga.com](https://www.benzinga.com/money/ethereum-eth-price-predictions?utm_source=openai))

- **The Economic Times** reports that Ethereum's price in 2025 is expected to range between $5,200 and $8,666, influenced by market maturation and post-halving price adjustments. ([m.economictimes.com](https://m.economictimes.com/markets/cryptocurrency/ethereum-price-prediction-forecast-for-2024-to-2030-mudrex-research-team/amp_articleshow/112462865.cms?utm_source=openai))

These forecasts are based on factors such as network upgrades, the growth of decentralized finance (DeFi), and broader blockchain adoption. However, it's important to note that cryptocurrency markets are highly volatile, and actual prices may vary due to market dynamics and unforeseen events. 

Take a look at the trace

https://platform.openai.com/traces

## Our Second Agent: Planner Agent

Given a query, come up with 5 ideas for web searches that could be run.

Use Structured Outputs as our way to ensure the Agent provides what we need.

In [5]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 5

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# We use Pydantic objects to describe the Schema of the output

class WebSearchItem(BaseModel):
    reason: str
    "Your reasoning for why this search is important to the query."

    query: str
    "The search term to use for the web search."


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query."""

# We pass in the Pydantic object to ensure the output follows the schema

planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4.1-mini",
    output_type=WebSearchPlan,
)

In [7]:

message = "What is the price prediction for ETH for May30th and June 31st of 2025?"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    pprint(result.final_output)

WebSearchPlan(searches=[WebSearchItem(reason='To find expert price predictions for Ethereum (ETH) for specific future dates in 2025.', query='Ethereum price prediction May 30 2025'), WebSearchItem(reason='To gather additional predictions and forecasts for ETH price in mid-2025 for comparison.', query='ETH price forecast June 2025'), WebSearchItem(reason='To look for any detailed analyses or reports providing ETH price targets specifically for June 31 2025.', query='Ethereum price target June 31 2025'), WebSearchItem(reason='To explore cryptocurrency news websites or analysts with long-term ETH price insights.', query='Ethereum long term price prediction 2025'), WebSearchItem(reason='To find data from cryptocurrency prediction platforms or algorithms for ETH prices on specified dates.', query='ETH price prediction algorithms May June 2025')])


## Our Third Agent: Writer Agent

Take the results of internet searches and make a report

In [8]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str
    """A short 2-3 sentence summary of the findings."""

    markdown_report: str
    """The final report"""

    follow_up_questions: list[str]
    """Suggested topics to research further"""


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

## Our Fourth Agent: push notification

Just to show how easy it is to make a tool!

I'm using a nifty product called PushOver - to set this up yourself, visit https://pushover.net

In [9]:
@function_tool
def push(message: str):
    """Send a push notification with this brief message"""
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)
    return {"status": "success"}

In [10]:
push

FunctionTool(name='push', description='Send a push notification with this brief message', params_json_schema={'properties': {'message': {'title': 'Message', 'type': 'string'}}, 'required': ['message'], 'title': 'push_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10dd7f1a0>, strict_json_schema=True)

In [11]:
INSTRUCTIONS = """You are a member of a research team and will be provided with a short summary of a report.
When you receive the report summary, you send a push notification to the user using your tool, informing them that research is complete,
and including the report summary you receive"""


push_agent = Agent(
    name="Push agent",
    instructions=INSTRUCTIONS,
    tools=[push],
    model="gpt-4.1-mini",
    model_settings=ModelSettings(tool_choice="required")
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [12]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and send a push notification

In [13]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_push(report: ReportData):
    """ Use the push agent to send a notification to the user """
    print("Pushing...")
    result = await Runner.run(push_agent, report.short_summary)
    print("Push sent")
    return report

### Showtime!

In [ ]:
query ="What is the prediction of ETH prices for May 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_push(report)  
    print("Hooray!")
display(Markdown(report.markdown_report))

Starting research...
Planning searches...
Will perform 5 searches
Searching...
Finished searching
Thinking about report...


### As always, take a look at the trace

https://platform.openai.com/traces